<a href="https://colab.research.google.com/github/gkckdals3/ML_/blob/main/%EA%B2%BD%EC%A7%84%EB%8C%80%ED%9A%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import cv2

In [37]:
import glob
from google.colab import drive
import pandas as pd
drive.mount('/gdrive', force_remount=True)
#train 타겟값 target에 저장
target_table=pd.read_csv('/gdrive/MyDrive/SyntekaBio/train_output.csv').to_numpy()
target=target_table[:,1:].astype(float)
train_data =(sorted(glob.glob('/gdrive/MyDrive/SyntekaBio/train/PNG/*.png')))


Mounted at /gdrive


In [38]:
#전체 데이터 불러오기/데이터 10분에 1로 축소
for i in range (len(train_data)):
  train_data[i]=cv2.imread(train_data[i])
  train_data[i]=cv2.resize(train_data[i],dsize=(128, 128),interpolation=cv2.INTER_AREA)
  train_data[i]=train_data[i].astype(np.float32)/255
train_scaled=np.array(train_data)
train_scaled.shape

(501, 128, 128, 3)

In [39]:
from tensorflow import keras
from sklearn.model_selection import train_test_split

train_scaled.reshape(-1,128,128,3)

array([[[[1.        , 1.        , 0.84313726],
         [1.        , 1.        , 0.84313726],
         [1.        , 1.        , 0.84313726],
         ...,
         [1.        , 1.        , 0.84313726],
         [1.        , 1.        , 0.84313726],
         [1.        , 0.        , 0.        ]],

        [[1.        , 1.        , 0.84313726],
         [1.        , 0.        , 0.        ],
         [1.        , 1.        , 0.84313726],
         ...,
         [1.        , 1.        , 0.84313726],
         [1.        , 1.        , 0.84313726],
         [1.        , 1.        , 0.84313726]],

        [[1.        , 1.        , 0.84313726],
         [1.        , 1.        , 0.84313726],
         [1.        , 1.        , 0.84313726],
         ...,
         [1.        , 0.        , 0.        ],
         [1.        , 1.        , 0.84313726],
         [1.        , 1.        , 0.84313726]],

        ...,

        [[1.        , 1.        , 0.84313726],
         [1.        , 1.        , 0.84313726]

In [40]:
from sklearn.model_selection import train_test_split

train_scaled,val_scaled,train_target,val_target=train_test_split(train_scaled,target,test_size=0.2)
print('학습 데이터 shape : ', train_scaled.shape)
print('검증 데이터 shape : ', val_scaled.shape)
print('학습 타겟 shape : ', train_target.shape)
print('검증 타겟 shape : ', val_target.shape)

학습 데이터 shape :  (400, 128, 128, 3)
검증 데이터 shape :  (101, 128, 128, 3)
학습 타겟 shape :  (400, 1)
검증 타겟 shape :  (101, 1)


In [41]:
model = keras.Sequential()
model.add(keras.layers.Conv2D(32, kernel_size=(3,3), activation='relu', kernel_initializer='he_uniform', input_shape=(128,128,3)))
model.add(keras.layers.MaxPooling2D(2))
model.add(keras.layers.BatchNormalization(center=True, scale=True))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Conv2D(64, kernel_size=(3,3), activation='relu', kernel_initializer='he_uniform'))
model.add(keras.layers.MaxPooling2D(2))
model.add(keras.layers.BatchNormalization(center=True, scale=True))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(100, activation='relu', kernel_initializer='he_uniform'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(1, activation='softmax'))

In [42]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 126, 126, 32)      896       
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 63, 63, 32)        0         
_________________________________________________________________
batch_normalization_10 (Batc (None, 63, 63, 32)        128       
_________________________________________________________________
dropout_10 (Dropout)         (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 61, 61, 64)        18496     
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 30, 30, 64)        0         
_________________________________________________________________
batch_normalization_11 (Batc (None, 30, 30, 64)       

In [43]:
import tensorflow
model.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.Adam(lr=0.001),
              metrics=['accuracy'])
print(train_scaled.shape)
print(train_target.shape)

(400, 128, 128, 3)
(400, 1)


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [44]:
history = model.fit(
            train_scaled, train_target, 
            validation_data = (val_scaled,val_target),
            verbose = 1,
            batch_size = 32,
            epochs = 30
                    )

Epoch 1/30
12/12 [==============================] - 5s 410ms/step - loss: 0.0000e+00 - accuracy: 0.6600 - val_loss: 0.0000e+00 - val_accuracy: 0.6733
Epoch 2/30
12/12 [==============================] - 5s 398ms/step - loss: 0.0000e+00 - accuracy: 0.6600 - val_loss: 0.0000e+00 - val_accuracy: 0.6733
Epoch 3/30
12/12 [==============================] - 5s 384ms/step - loss: 0.0000e+00 - accuracy: 0.6600 - val_loss: 0.0000e+00 - val_accuracy: 0.6733
Epoch 4/30
12/12 [==============================] - 5s 390ms/step - loss: 0.0000e+00 - accuracy: 0.6600 - val_loss: 0.0000e+00 - val_accuracy: 0.6733
Epoch 5/30
12/12 [==============================] - 5s 390ms/step - loss: 0.0000e+00 - accuracy: 0.6600 - val_loss: 0.0000e+00 - val_accuracy: 0.6733
Epoch 6/30
12/12 [==============================] - 5s 390ms/step - loss: 0.0000e+00 - accuracy: 0.6600 - val_loss: 0.0000e+00 - val_accuracy: 0.6733
Epoch 7/30
12/12 [==============================] - 5s 389ms/step - loss: 0.0000e+00 - accuracy: 0.6

KeyboardInterrupt: ignored